# Detection  of demaged parts on conveer


 
 

### Motivation :

In this work we will make the computer vision program what can recognize normal parts 'ok' and demaged parts 'nok', sand signal for rlrctromechanics part on conveyor and srparete two tipes of parts and count the damaged parts for statistics. 

As a result we must get redusing a labor cost in conveyor production.

### Introduction

To get result we nead to make next steps:
 - 1. Make our oun dataset with labling a both classes ('ok', 'nok');
 - 2. Chose neural network, way of training and presets to get sutable result with small dataset.
 - 3. Make the working program to detect on a 'nok' parts and signalise to electro-mechanical part to separate 'ok'       and 'nok' pants with counter to make statistics for Quality Engineer (QE).

### Description

1. Dataset:
Images for dataset i make from digital camera with IR backlight in constant conditions and distance, thet why i get the same quality of pictures for training and in real application, but there is a limit number of a parts, thet why the size of a dataset is not large. (1000+ pictures. 370 - 'ok', 370- 'nok', and 270 - 4 mixed parts in a row in ewery picture)
2. Type of neural network: 
I tried a few tipes of neural networks for example, but SNN can't handle it with small dataset and diferent positions of parts with diferent angles. CNN like YOLOv8n did it well in a real time , so i chose it. 
3. Training of neural network: 
I chosed training in a 4 stage with 250 epochs with hiper functions with augmentation

# Training program

#### Part 0: Import necessary libraries

In [ ]:
# Import necessary libraries
import ast
import cv2
import numpy as np
import os
import pandas as pd
import random
import torch
import yaml
from collections import Counter
from tqdm.notebook import tqdm
from ultralytics import YOLO
from matplotlib import pyplot as plt
from PIL import Image
from pathlib import Path

# Configure matplotlib
plt.rcParams['figure.figsize'] = [20, 15]


#### Part 1: Verify CUDA Availability

In [ ]:
# Verify CUDA availability
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
print(f"CUDA available: {torch.cuda.is_available()}")

#### Part 2: Define Paths and Initialize Variables

In [ ]:

DATA_PATH = "/home/CV/Computer-Vision-v2-master/lesson_18/data/detect.v2i.yolov8_1000/data.yaml"
MODEL_WEIGHTS = "yolov8n.pt"  # Pretrained weights
PROJECT_DIR = "/home/CV/Computer-Vision-v2-master/lesson_18/data/results"
EXPERIMENT_NAME = "yolov8_experiment_progressive"
EPOCHS_PER_STAGE = 250  # Number of epochs per stage


# Finding the latest experiment directory


def find_latest_experiment(project_dir, experiment_name):
    project_path = Path(project_dir)
    experiment_dirs = sorted(project_path.glob(f"{experiment_name}*"), key=os.path.getmtime, reverse=True)
    if not experiment_dirs:
        raise FileNotFoundError(f"No experiment directories found with name starting with '{experiment_name}' in {project_dir}")
    return experiment_dirs[0]


# Runing a training stage with augmentations


def run_training_stage(stage_num, initial_weights, epochs=250):
    current_experiment_name = f"{EXPERIMENT_NAME}_stage{stage_num}"
    
    # Initializing the model with the initial_weights
    model = YOLO(initial_weights)
    
    # Train the model. We add additional augmentation parameters:
    # - degrees: rotate images by up to ±10 degrees
    # - fliplr: probability of horizontal flip
    # - flipud: probability of vertical flip
    # - scale: scale images by ±50%
    model.train(
        data=DATA_PATH,
        epochs=epochs,
        imgsz=640,
        batch=8,
        name=current_experiment_name,
        workers=8,
        project=PROJECT_DIR,
        device=device,
        verbose=True,
        save=True,
        save_period=-1,
        optimizer='auto',
        seed=0,
        deterministic=True,
        resume=False,
        cache=False,
        augment=True,
        multi_scale=True,
        amp=True,
        degrees=10,
        fliplr=0.5,
        flipud=0.5,
        scale=0.5
    )

    # Finding the best model from this stage
    latest_experiment_dir = find_latest_experiment(PROJECT_DIR, current_experiment_name)
    best_model_path = latest_experiment_dir / "weights" / "best.pt"

    if not best_model_path.exists():
        raise FileNotFoundError(f"The best.pt file was not found at {best_model_path} after stage {stage_num}.")

    print(f"Stage {stage_num} best model path: {best_model_path}")
    return best_model_path

#### Part 3: Run Multiple Stages of Training

In [ ]:
# Stage 1: Start from the pretrained model weights
best_model_path_stage1 = run_training_stage(stage_num=1, initial_weights=MODEL_WEIGHTS, epochs=EPOCHS_PER_STAGE)

# Stage 2: Start from best model of stage 1
best_model_path_stage2 = run_training_stage(stage_num=2, initial_weights=best_model_path_stage1, epochs=EPOCHS_PER_STAGE)

# Stage 3: Start from best model of stage 2
best_model_path_stage3 = run_training_stage(stage_num=3, initial_weights=best_model_path_stage2, epochs=EPOCHS_PER_STAGE)

# Stage 4: Start from best model of stage 3
best_model_path_stage4 = run_training_stage(stage_num=4, initial_weights=best_model_path_stage3, epochs=EPOCHS_PER_STAGE)

# Stage 5: Start from best model of stage 4
best_model_path_stage5 = run_training_stage(stage_num=5, initial_weights=best_model_path_stage4, epochs=EPOCHS_PER_STAGE)

# After finishing all stages, the best model from the last stage is your final model.
final_best_model_path = best_model_path_stage5
print(f"Final best model after 5 stages: {final_best_model_path}")


#### Part 4: Run Inference on the Test Set with the Final Model

In [ ]:
best_model = YOLO(final_best_model_path)
TEST_IMAGES_DIR = "/home/CV/Computer-Vision-v2-master/lesson_18/data/detect.v2i.yolov8_1000/test/images"
TEST_PREDICTIONS_DIR = Path(PROJECT_DIR) / f"{EXPERIMENT_NAME}_test_predictions"

predictions = best_model.predict(
    source=TEST_IMAGES_DIR,
    save=True,
    save_txt=True,
    save_conf=True,
    save_crop=True,
    imgsz=640,
    conf=0.7,
    iou=0.75,
    device=device,
    exist_ok=True,
    project=PROJECT_DIR,
    name=f"{EXPERIMENT_NAME}_test_predictions"
)

print(f"Inference completed. Predictions saved to {TEST_PREDICTIONS_DIR}")

#### Part 5: Evaluate the Model on the Test Set

In [ ]:
TEMP_TEST_DATA_YAML = "/home/CV/Computer-Vision-v2-master/lesson_18/data/detect.v2i.yolov8_1000/test_data.yaml"

with open(DATA_PATH, 'r') as f:
    data_yaml = yaml.safe_load(f)

test_data_yaml = {
    'train': data_yaml.get('train', '../train/images'),
    'val': 'test/images',
    'test': 'test/images',
    'nc': data_yaml['nc'],
    'names': data_yaml['names']
}

# Optionally, include roboflow metadata if present
if 'roboflow' in data_yaml:
    test_data_yaml['roboflow'] = data_yaml['roboflow']

with open(TEMP_TEST_DATA_YAML, 'w') as f:
    yaml.dump(test_data_yaml, f)

print(f"Temporary test data.yaml created at {TEMP_TEST_DATA_YAML}")

test_results = best_model.val(data=TEMP_TEST_DATA_YAML)


#### Part 6: Print Evaluation Metrics

In [ ]:
print("\nTest Set Evaluation Metrics:")
try:
    metrics_dict = test_results.results_dict
    for key, value in metrics_dict.items():
        print(f"{key}: {value}")
except AttributeError as e:
    print("Error accessing metrics:", e)
    print("Available keys:", test_results.keys())
    print("Maps:", test_results.maps)
    print("Fitness:", test_results.fitness)

#### Part 8: Examples of Predictions

In [ ]:
if predictions:
    for i, result in enumerate(predictions[:3]):
        print(f"\nPrediction {i+1} Details:")
        print(f"Image shape: {result.orig_shape}")
        print("Detections:")
        for box in result.boxes:
            cls_id = int(box.cls)
            confidence = float(box.conf)
            x1, y1, x2, y2 = box.xyxy[0].tolist()
            class_name = data_yaml['names'][cls_id] if cls_id < len(data_yaml['names']) else "unknown"
            print(f" - {class_name} at [{x1:.2f}, {y1:.2f}, {x2:.2f}, {y2:.2f}] (conf: {confidence:.2f})")

val: Scanning /home/st/CV/Computer-Vision-v2-master/lesson_18/data/detect.v2i.yo
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all        105        180      0.984      0.994      0.989      0.826
                   nok         68         68      0.985      0.998      0.984      0.799
                    ok         64        112      0.983      0.991      0.995      0.852
Speed: 0.2ms preprocess, 4.0ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/val15

Test Set Evaluation Metrics:
metrics/precision(B): 0.9843869703645236
metrics/recall(B): 0.9943543397243358
metrics/mAP50(B): 0.989431288819876
metrics/mAP50-95(B): 0.8258556818617497
fitness: 0.8422132425575625


Prediction 1 Details:
Image shape: (640, 640)
Detections:
 - ok at [317.43, 138.53, 351.72, 183.33] (conf: 0.89)

Prediction 2 Details:
Image shape: (640, 640)
Detections:
 - ok at [283.14, 244.39, 317.99, 289.41] (conf: 0.90)

Prediction 3 Details:
Image shape: (640, 640)
Detections:
 - ok at [272.64, 384.43, 308.51, 428.58] (conf: 0.90)


# Detection program

#### Import necessary libraries

In [ ]:
import cv2
import torch
import numpy as np
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
from ultralytics import YOLO
import time
import sys

#### Configuration

In [ ]:
BEST_MODEL_PATH = "/home/st/CV/Computer-Vision-v2-master/lesson_18/data/results/yolov8_experiment_progressive_stage402/weights/best.pt"  # Path to your trained YOLOv8 model
VIDEO_INPUT_PATH = "/home/st/CV/Computer-Vision-v2-master/lesson_18/data/detect.v1i.yolov8/video/20241222_20_16_04.mp4"  # Input .mp4 video path
VIDEO_OUTPUT_PATH = "/home/st/CV/Computer-Vision-v2-master/lesson_18/data/detect.v1i.yolov8/output_video/output_video.mp4"  # Output .mp4 video path
CONF_THRESHOLD = 0.80  # Confidence threshold for detections
IOU_THRESHOLD = 0.85   # IoU threshold for Non-Max Suppression
FONT_PATH = None        # Optional: Path to a .ttf font file for labels (e.g., "/path/to/font.ttf")

#### Counter Configuration

In [ ]:
MAX_COUNT = 100000       # Maximum count value
counter = 0              # Initialize counter
square_on = False        # Flag to indicate if the red square is currently displayed

#### Initializing Device

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
print(f"CUDA available: {torch.cuda.is_available()}")

#### Loading the YOLOv8 Model

In [ ]:
try:
    model = YOLO(BEST_MODEL_PATH)
    model.to(device)
    names = model.names  # Class names from the model
    print("YOLOv8 model loaded successfully.")
except Exception as e:
    print(f"Error loading YOLOv8 model: {e}")
    sys.exit(1)


#### Define Class Color Mapping

In [ ]:
# Define colors for each class. Modify this dictionary based on your actual class names.
# For example, if your classes are ['nok', 'ok'], map them accordingly.
class_color_map = {
    'nok': 'red',
    'ok': 'green'
}

#### Drawing a Bounding Boxes and Signalization

In [ ]:
def draw_boxes_on_frame(frame, boxes, names, class_color_map, font_path=None):
    """
    Draws bounding boxes and labels on the frame. Additionally, draws a vertical line and
    a red square in the top-left corner if a 'nok' object crosses the line.
    
    Parameters:
    - frame (numpy.ndarray): The image frame in BGR format.
    - boxes (BoxList): The detected bounding boxes.
    - names (list): List of class names.
    - class_color_map (dict): Mapping from class names to colors.
    - font_path (str): Path to the .ttf font file.
    
    Returns:
    - numpy.ndarray: The annotated frame in BGR format.
    - bool: Flag indicating if a 'nok' object has crossed the line.
    """
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    draw_obj = ImageDraw.Draw(img)
    
    # Get image dimensions
    width, height = img.size
    x_line = width / 2  # Vertical line in the center
    
    # Draw the vertical line (top to bottom) with 2-pixel thickness
    line_color = 'blue'  # Color for the vertical line
    draw_obj.line([(x_line, 0), (x_line, height)], fill=line_color, width=2)
    
    # Flag to check if any 'nok' object crosses the line
    nok_crossed = False
    
    # Load a font if FONT_PATH is provided; else use default
    if font_path and Path(font_path).exists():
        try:
            font = ImageFont.truetype(font_path, 20)
        except IOError:
            print("Custom font not found. Using default font.")
            font = ImageFont.load_default()
    else:
        font = ImageFont.load_default()

    # Iterate through each detected box
    for box in boxes:
        cls_id = int(box.cls)
        conf = float(box.conf)
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        class_name = names[cls_id] if cls_id < len(names) else "unknown"
        label = f"{class_name} {conf:.2f}"
        
        # Determine color based on class
        color = class_color_map.get(class_name, 'blue')  # Default to blue if class not mapped
        
        # Draw bounding box
        draw_obj.rectangle([x1, y1, x2, y2], outline=color, width=3)
        
        # Calculate text size using a try-except block for compatibility
        try:
            text_w, text_h = draw_obj.textsize(label, font=font)
        except AttributeError:
            # For newer Pillow versions
            bbox = draw_obj.textbbox((0, 0), label, font=font)
            text_w = bbox[2] - bbox[0]
            text_h = bbox[3] - bbox[1]
        
        # Draw label background
        draw_obj.rectangle([x1, y1 - text_h - 4, x1 + text_w + 4, y1], fill=color)
        # Draw label text
        draw_obj.text((x1 + 2, y1 - text_h - 2), label, fill="white", font=font)
        
        # Check if 'nok' object crosses the vertical line
        if class_name == 'nok':
            # Define a small tolerance (e.g., 2 pixels)
            tolerance = 2
            if (x1 - tolerance) <= x_line <= (x2 + tolerance):
                nok_crossed = True

    # If any 'nok' object crosses the line, draw a red square in the top-left corner
    if nok_crossed:
        square_size = 50
        square_color = 'red'
        draw_obj.rectangle([0, 0, square_size, square_size], fill=square_color)
    
    # Convert back to BGR for OpenCV
    annotated_frame = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    
    return annotated_frame, nok_crossed


#### Initializinf Video Capture

In [ ]:
def initialize_video_capture(video_path):
    """
    Initializes the video capture object for the given video file.
    
    Parameters:
    - video_path (str): Path to the input video file.
    
    Returns:
    - cap (cv2.VideoCapture): The video capture object.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"Failed to open video file: {video_path}")
    return cap

#### Real-Time Video Processing with Counter

In [ ]:
def process_video_file():
    """
    Processes the input video file frame by frame, performing object detection,
    counting 'nok' object crossings, and saving the annotated video.
    """
    global counter, square_on  # To modify the global counter and square_on variables
    cap = initialize_video_capture(VIDEO_INPUT_PATH)
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps == 0:
        fps = 30  # Default to 30 if unable to get FPS
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"Video Properties:\n - Resolution: {width}x{height}\n - FPS: {fps}\n - Total Frames: {total_frames}")
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    output_dir = Path(VIDEO_OUTPUT_PATH).parent
    output_dir.mkdir(parents=True, exist_ok=True)
    out = cv2.VideoWriter(VIDEO_OUTPUT_PATH, fourcc, fps, (width, height))
    
    frame_count = 0
    start_time = time.time()
    print("Starting video processing...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # End of video
        frame_count += 1
        
        # Run inference on the frame
        results = model.predict(
            frame,
            conf=CONF_THRESHOLD,
            iou=IOU_THRESHOLD,
            device=device,
            verbose=False
        )
        
        dets = results[0]  # detections for this frame
        annotated_frame, nok_crossed = draw_boxes_on_frame(frame, dets.boxes, names, class_color_map, FONT_PATH)
        
        # Update counter and square_on flag
        if nok_crossed and not square_on:
            counter += 1
            square_on = True
            print(f"'nok' object crossed the line. Current count: {counter}")
            if counter >= MAX_COUNT:
                print(f"Maximum count of {MAX_COUNT} reached.")
                counter = MAX_COUNT  # Prevent counter from exceeding MAX_COUNT
        elif not nok_crossed and square_on:
            square_on = False  # Reset the flag when no crossing is detected
        
        # Draw the counter at the top center of the frame
        img_pil = Image.fromarray(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))
        draw = ImageDraw.Draw(img_pil)
        
        # Load a font for the counter
        if FONT_PATH and Path(FONT_PATH).exists():
            try:
                font = ImageFont.truetype(FONT_PATH, 25)
            except IOError:
                print("Custom font not found. Using default font.")
                font = ImageFont.load_default()
        else:
            font = ImageFont.load_default()
        
        counter_text = f"Count: {counter}"
        try:
            text_w, text_h = draw.textsize(counter_text, font=font)
        except AttributeError:
            # For newer Pillow versions
            bbox = draw.textbbox((0, 0), counter_text, font=font)
            text_w = bbox[2] - bbox[0]
            text_h = bbox[3] - bbox[1]
        
        # Calculate position: top center
        text_x = (width - text_w) / 2
        text_y = 10  # 10 pixels from the top
        
        # Draw a solid rectangle behind the text for better visibility
        draw.rectangle([text_x - 5, text_y - 5, text_x + text_w + 5, text_y + text_h + 5], fill=(0, 0, 0))
        # Draw the counter text
        draw.text((text_x, text_y), counter_text, fill="yellow", font=font)
        
        # Convert back to BGR for OpenCV
        annotated_frame_with_counter = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)
        
        # Write the annotated frame to the output video
        out.write(annotated_frame_with_counter)
        
        # Optional: Display progress every 100 frames
        if frame_count % 100 == 0:
            elapsed_time = time.time() - start_time
            print(f"Processed {frame_count}/{total_frames} frames in {elapsed_time:.2f} seconds.")
    
    # Release everything if job is finished
    cap.release()
    out.release()
    total_time = time.time() - start_time
    print(f"\nProcessing completed. Output saved to: {VIDEO_OUTPUT_PATH}")
    print(f"Total Frames Processed: {frame_count}")
    print(f"Total Time Taken: {total_time:.2f} seconds")
    print(f"Average FPS: {frame_count / total_time:.2f}")

#### Main Execution

In [ ]:
if __name__ == "__main__":
    try:
        process_video_file()
    except Exception as e:
        print(f"An error occurred: {e}")

#### Print: 

Using device: cuda
CUDA available: True
YOLOv8 model loaded successfully.
Video Properties:
 - Resolution: 640x480
 - FPS: 12.53066199116487
 - Total Frames: 199
Starting video processing...
'nok' object crossed the line. Current count: 1
'nok' object crossed the line. Current count: 2
'nok' object crossed the line. Current count: 3
'nok' object crossed the line. Current count: 4
'nok' object crossed the line. Current count: 5
'nok' object crossed the line. Current count: 6
Processed 100/199 frames in 2.06 seconds.
'nok' object crossed the line. Current count: 7
'nok' object crossed the line. Current count: 8
'nok' object crossed the line. Current count: 9

Processing completed. Output saved to: /home/st/CV/Computer-Vision-v2-master/lesson_18/data/detect.v1i.yolov8/output_video/output_video.mp4
Total Frames Processed: 199
Total Time Taken: 4.02 seconds
Average FPS: 49.47